# Membres de l'équipe:

# Etudiant 1:

* Nom: **Assani Amate**
* Prénom:**Ousmane**
* Code permanent **ASSO8431004**
# Etudiant 2:

* Nom: **SIL BEKONA**
* Prénom: **Alix**
* Code permanent: **SILA063295090**

# Projet 2 : Traitement des données massives

# Partie 1 : Données

# Question 1 :  Quelle est l'origine des données (lien, source) ?

# Réponse :

*  Lien: https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset/data
*  Source: Recueilli par Cai-Nicolas Ziegler au cours d'une collecte de 4 semaines (août/septembre 2004) auprès de la communauté Book-Crossing avec l'aimable autorisation de Ron Hornbaker, CTO de Humankind Systems. Contient 278 858 utilisateurs (anonymisés mais avec des informations démographiques) fournissant 1 149 780 évaluations (explicites/implicites) sur 271 379 livres.

# Question 2 : Quel est le contexte du jeu de données ?


# Réponse :

Au cours des dernières décennies, avec l'essor de Youtube, Amazon, Netflix et bien d'autres services web du même type, les systèmes de recommandation ont pris une place de plus en plus importante dans nos vies. Du e-commerce (proposer aux acheteurs des articles susceptibles de les intéresser) à la publicité en ligne (proposer aux utilisateurs des contenus adaptés, correspondant à leurs préférences), les systèmes de recommandation sont aujourd'hui incontournables dans nos parcours en ligne quotidiens.
Les systèmes de recommandation sont vraiment essentiels dans certains secteurs car ils peuvent générer d’énormes revenus lorsqu’ils sont efficaces ou être un moyen de se démarquer considérablement de la concurrence. Pour preuve de l’importance des systèmes de recommandation, nous pouvons mentionner qu’il y a quelques années, Netflix a organisé un concours (le « prix Netflix ») dont l’objectif était de produire un système de recommandation plus performant que son propre algorithme avec un prix d’un million de dollars à gagner.

**Contenu**:
L'ensemble de données Book-Crossing comprend 3 fichiers.
*  Users:
Contient les utilisateurs. Notez que les identifiants d'utilisateur (User-ID) ont été anonymisés et correspondent à des entiers. Des données démographiques sont fournies (emplacement, âge) si elles sont disponibles. Sinon, ces champs contiennent des valeurs NULL.
*  Books:
Les livres sont identifiés par leur ISBN respectif. Les ISBN non valides ont déjà été supprimés de l'ensemble de données. De plus, certaines informations basées sur le contenu sont fournies (Titre du livre, Auteur du livre, Année de publication, Éditeur), obtenues à partir d'Amazon Web Services. Notez que dans le cas de plusieurs auteurs, seul le premier est fourni. Des URL menant à des images de couverture sont également fournies, apparaissant sous trois formes différentes (Image-URL-S, Image-URL-M, Image-URL-L), c'est-à-dire petite, moyenne, grande. Ces URL pointent vers le site Web d'Amazon.
*  Ratings:
Contient les informations de notation du livre. Les notes (Book-Rating) sont soit explicites, exprimées sur une échelle de 1 à 10 (les valeurs les plus élevées dénotant une appréciation plus élevée), soit implicites, exprimées par 0.

* Remarque: Les trois fichiers contiennent beaucoup des lignes (plus de 200 000 chacun) et qu'on a eu du mal à les intégrer dans Neo4J car ça prenait trop de temps, nous avons sélectionné que les 10 000 premières lignes pour chaque fichier.

# Prétraitement des données
* Problème: Dans le fichier Users, la colonne **Age** contient beaucoup de valeurs manquantes pour environ **110762** utilisateurs (environ 40 % des données). La colonne **Location** aussi contient des valeurs manquantes.
* Solution proposée: D'abord la colonne Location a été divisée en trois colonnes à savoir la colonne **City**, **State** et **Country** car elle contient les informations de la ville, l'état et du pays de l'utilisateur en question séparées par des virgules. Maintenant pour les lignes (utilisateurs) dont l'age est manquant et aussi l'état et le pays sont manquants, ces derniers sont supprimés. Pour les utilisateurs dont seulement l'age est manquant, nous avaons utilisé un modèle de machine learning (Random forest regression) pour prédire l'age de l'utilisateur en fonction des autres colonnes (features).
Le code contenant toutes les étapes du prétraitement (en python) est contenu dans le fichier Prétraitement_données.ipynb
Le fichier Users.csv est devenu Users_cleaned.csv après le prétraitement et c'est ce dernier (Users_cleaned.csv) qui sera utilisé pour la suite du prijet.
Nous avons aussi supprimer les colonnes Image-URL-S, Image-URL-M et Image-URL-L contenant les urls de ces livres car elles sont pas utiles pour notre travail.

# Partie 2 : Chargement des données dans Neo4J

# Question 1 : Quelles données chargeons-nous dans neo4j?

# Réponse :
Nous chargeons le dataset Book Recommendation composé de trois fichiers (Books.csv, Users.csv et Ratings.csv) comme mentionné dans la réponse à la question 2 de la partie 1.

# Question 1 : Faisons-nous des traitements/modifications lors du chargement?

# Réponse :
Non pas de traitements ou modifications lors du chargement des données.

# Chargement des données en utilisant Neo4J uniquement.

# Requête pour le chargement du fichier Books.csv

```
LOAD CSV WITH HEADERS FROM 'file:///Books_subset.csv' AS row
WITH row
WHERE row.`Book-Title` IS NOT NULL AND row.`Book-Author` IS NOT NULL
MERGE (b:Book {ISBN: row.ISBN})
SET b.title = row.`Book-Title`,
    b.author = row.`Book-Author`,
    b.year = toInteger(row.`Year-Of-Publication`),
    b.publisher = row.Publisher;
```

# Description des étapes de la requête ci-dessus (Fichiers Books_subset.csv)

# 1. Charger les données CSV :
LOAD CSV WITH HEADERS FROM 'file:///Books.csv' AS row

* LOAD CSV WITH HEADERS: Cette commande lit les données du fichier Books.csv dans le répertoire import de Neo4j, en interprétant la première ligne comme des en-têtes.
* AS row: Chaque ligne du fichier est traitée comme un objet row, où les colonnes peuvent être référencées par leurs noms (par exemple, row.ISBN, row.Book-Title)

# 2. Filtrer les lignes avec WITH et WHERE:
WITH row
WHERE row.`Book-Title` IS NOT NULL AND row.`Book-Author` IS NOT NULL
* WITH row: Transmet chaque ligne (ou variable) à l'étape suivante.
* WHERE row.`Book-Title` IS NOT NULL : Exclut les lignes où la colonne Book-Title est vide (NULL).
* AND row.`Book-Author` IS NOT NULL : Exclut également les lignes où la colonne Book-Author est vide.


# 2. Identifier ou créer un nœud Book :
MERGE (b:Book {ISBN: row.ISBN})

* MERGE:  Cherche un nœud Book avec une propriété ISBN correspondant à row.ISBN.
   * Si un tel nœud existe déjà, il le réutilise.
   * Sinon, il crée un nouveau nœud avec l'ISBN donné.
* (b:Book): Définit ou utilise un nœud avec le label Book.
* {ISBN: row.ISBN}: Associe l'ISBN comme identifiant unique pour ce nœud.
# Mettre à jour ou définir les propriétés du nœud Book
SET b.title = row.`Book-Title`,
    b.author = row.`Book-Author`,
    b.year = toInteger(row.`Year-Of-Publication`),
    b.publisher = row.Publisher';

* SET: Ajoute ou met à jour les propriétés d'un nœud. Dans ce cas, les propriétés du nœud b sont définies avec les valeurs provenant de row.

Propriétés mises à jour :
1.  b.title = row.`Book-Title`: Définit la propriété title avec la valeur de la colonne Book-Title.
2.  b.author = row.`Book-Author`: Définit la propriété author avec la valeur de la colonne Book-Author.
3. b.year = toInteger(row.`Year-Of-Publication`): Convertit la colonne Year-Of-Publication en entier avec toInteger. Et définit la propriété year.4. b.publisher = row.Publisher: Définit la propriété publisher avec la valeur de la colonne Publisher.


# Requête pour le chargement du fichier Users_cleaned.csv

```
LOAD CSV WITH HEADERS FROM 'file:///Users_cleaned.csv' AS row
MERGE (u:user {userId: row.`User-ID`})
SET u.userAge = row.Age,
    u.userCity = row.City,
    u.userState = row.State,
    u.userCountry = row.Country;
```

# Description des étapes de la requête ci-dessus (Fichier Users_cleaned.csv)

# 1. Charger le fichier CSV dans Neo4j
LOAD CSV WITH HEADERS FROM 'file:///Users_cleaned.csv' AS row

* LOAD CSV WITH HEADERS: Charge le fichier Users_cleaned.csv depuis le répertoire import de Neo4j.
* FROM 'file:///Users_cleaned.csv': Spécifie le chemin du fichier (doit être placé dans le répertoire import de Neo4j).
* AS row: Chaque ligne du fichier est traitée comme un objet row, où chaque colonne peut être référencée par son nom, par exemple row.`User-ID`.
# 2. Identifier ou créer un nœud user
MERGE (u:user {userId: row.`User-ID`})

* MERGE:
  * Cherche un nœud existant avec le label user et la propriété userId égale à row.`User-ID`.
  * Si un tel nœud existe, il le réutilise.
  * Sinon, il crée un nouveau nœud avec la propriété spécifiée.
    
* (u:user {userId: row.\User-ID`}):
  * Définit u comme un nœud avec le label user.
  * La propriété userId est utilisée comme clé unique pour identifier chaque utilisateur.

# 3. Ajouter ou mettre à jour les propriétés du nœud
SET u.userAge = row.Age,
    u.userCity = row.City,
    u.userState = row.State,
    u.userCountry = row.Country
    
* SET: Ajoute ou met à jour les propriétés du nœud user.
Propriétés définies :
1. u.userAge = row.Age :
   * Définit la propriété userAge avec la valeur de la colonne Age.
   * Exemple : Si Age = 25, alors userAge = 25.
2.  u.userCity = row.City :
   * Définit la propriété userCity avec la valeur de la colonne City.
     
3. u.userState = row.State :
   * Définit la propriété userState avec la valeur de la colonne State.
4. u.userCountry = row.Country :
   * Définit la propriété userCountry avec la valeur de la colonne Country.




# Requête pour le chargement du fichier Ratings.csv

```
LOAD CSV WITH HEADERS FROM 'file:///Ratings_subset.csv' AS row
WITH row
WHERE row.`User-ID` IS NOT NULL AND row.`User-ID` <> ""
MERGE (u:user {userId: toInteger(row.`User-ID`)});

```

# Description des étapes de la requête ci-dessus (Ratings_subset.csv)

# 1. Charger le fichier CSV
LOAD CSV WITH HEADERS FROM 'file:///Ratings_subset.csv' AS row

* LOAD CSV WITH HEADERS FROM 'file:///Ratings_subset.csv':
  * Charge les données du fichier CSV Ratings_subset.csv depuis le répertoire import de Neo4j.
  * La première ligne du fichier est interprétée comme en-tête de colonne.
* AS row:
  * Chaque ligne du fichier CSV est traitée comme un objet row.
  * Les colonnes du fichier peuvent être accédées en utilisant leur nom, par exemple, row.`User-ID`.
    
# 2. Passer les données à l’étape suivante
WITH row
* WITH row:
  * Transmet les données de chaque ligne à l'étape suivante.
  * Permet de manipuler et de filtrer les données avant leur utilisation dans d'autres opérations comme MERGE.

# 3. Filtrer les lignes valides
WHERE row.`User-ID` IS NOT NULL AND row.`User-ID` <> ""

* WHERE :
  * Filtre les lignes pour s'assurer que la colonne User-ID n'est ni NULL ni vide.
* row.`User-ID` IS NOT NULL :
  * Vérifie que la valeur dans la colonne User-ID n'est pas absente (non définie ou NULL).
* row.`User-ID` <> "" :
  * Vérifie que la valeur dans la colonne User-ID n'est pas une chaîne vide.
    
# 4. Fusionner ou créer un nœud utilisateur
MERGE (u:user {userId: toInteger(row.`User-ID`)});
* MERGE :
  * Cherche un nœud existant avec le label user et la propriété userId égale à toInteger(row.`\User-ID`).
  * Si un tel nœud existe déjà, il est réutilisé.
  * Sinon, un nouveau nœud est créé avec cette propriété.
* (u:user {userId: toInteger(row.`\User-ID`)}) :
  * Définit le nœud u avec:
    * Le label user : Identifie les nœuds comme des utilisateurs.
    * La propriété userId : Identifie l’utilisateur de manière unique.
    * La conversion toInteger(row.`User-ID`): Convertit la valeur de `User-ID` (qui est une chaîne dans le CSV) en entier.

# Étape suivante : Importer les relations RATED
Une fois les utilisateurs et les livres correctement importés, nous pouvons importer les relations entre eux (RATED) en utilisant une requête
comme celle-ci :

LOAD CSV WITH HEADERS FROM 'file:///Ratings_subset.csv' AS row
MATCH (u:user {userId: toInteger(row.`User-ID`)})
MATCH (b:Book {ISBN: row.ISBN})
MERGE (u)-[r:RATED]->(b)
SET r.rating = toInteger(row.`Book-Ratin`);





# Partie 3 : Recommandation

# Question : Quelle recommandation proposons-nous?

# Réponse :
Nous proposons un système de recommandation collaboratif (User-based) utilisant la similarité cosinus, autrement dit on va identifier les utilisateurs ayant une similarité élevée avec l'utilisateur cible. Et enfin recommander des livres bien notés par les utilisateurs similaires, mais non évalués par l'utilisateur cible.

# Ce qui est recommandé :
Les livres bien notés (très bon ratings) par les utilisateurs similaires à l'utilisant cible vont lui être recommandés (ces livres ne sont pas évalués par cet utilisateur).

# À qui s'adresse cette recommandation? :
À un utilisateur qui est similaire aux utilisateurs ayant bien notés les livres recommandés.

# Requêtes pour la recommandation

# 1. Représenter les évaluations des utilisateurs:
Cette requête retourne une liste de livres évalués et leurs notes pour chaque utilisateur.

MATCH (u:user)-[r:RATED]->(b:Book)
RETURN u.userId AS UserId, COLLECT({ISBN: b.ISBN, rating: r.rating}) AS Ratings
LIMIT 10;
# 2. Calculer la similarité cosinus entre utilisateurs:
Nous utilisons l'algorithme de similarité cosinus pour calculer les scores entre utilisateurs partageant des livres évalués. On choisit l'utilisateur cible en spécifiant son userID (243) et on essaie de trouver les utilisateurs qui lui sont similaires.

MATCH (u1:user)-[r1:RATED]->(b:Book)<-[r2:RATED]-(u2:user)
WHERE u1.userId =2435 AND u1 <> u2
WITH
    u1,
    u2,
    COLLECT(r1.rating * r2.rating) AS productRatings,
    COLLECT(r1.rating) AS ratings1,
    COLLECT(r2.rating) AS ratings2
WITH
    u1,
    u2,
    REDUCE(sum = 0, x IN productRatings | sum + x) AS dotProduct,
    SQRT(REDUCE(sum = 0, x IN ratings1 | sum + x^2)) AS magnitude1,
    SQRT(REDUCE(sum = 0, x IN ratings2 | sum + x^2)) AS magnitude2
WITH
    u1,
    u2,
    CASE WHEN magnitude1 * magnitude2 = 0 THEN 0 ELSE dotProduct / (magnitude1 * magnitude2) END AS similarity
RETURN u1.userId AS TargetUser, u2.userId AS SimilarUser, similarity
ORDER BY similarity DESC
LIMIT 5;

* Explication:
  * Trouve les utilisateurs ayant évalué les mêmes livres que l'utilisateur cible.
  * Calcule:
    * Produit scalaire des évaluations communes.
    * Magnitudes des vecteurs d'évaluations.
    * Similarité cosinus en combinant ces valeurs.

# 3. Recommander des livres des utilisateurs similaires
Après avoir trouvé les utilisateurs similaires, on recommande leurs livres bien notés mais non évalués par l'utilisateur cible.

MATCH (u1:user {userId: 243})-[:RATED]->(b:Book)
WITH COLLECT(b.ISBN) AS alreadyRated, u1
MATCH (u2:user)-[r:RATED]->(b2:Book)
WHERE NOT b2.ISBN IN alreadyRated AND u2 <> u1
WITH u2, b2, r.rating AS rating, u1
MATCH (u1)-[r1:RATED]->(b:Book)<-[r2:RATED]-(u2)
WITH b2, AVG(rating) AS AvgRating, COUNT(*) AS SimilarityScore
WHERE AvgRating > 7
ORDER BY SimilarityScore DESC, AvgRating DESC
RETURN b2.title AS RecommendedBook, b2.author AS Author, AvgRating, SimilarityScore
LIMIT 10;

* Explication:
  * Étape 1 : Trouver les livres déjà évalués par l'utilisateur cible.

    MATCH (u1:user {userId: 243})-[:RATED]->(b:Book)
    WITH COLLECT(b.ISBN) AS alreadyRated, u1

  * MATCH (u1:user {userId: 243})-[:RATED]->(b:Book) :
    * Identifie l'utilisateur cible (u1) avec userId = 243.
    * Trouve tous les livres (b) que cet utilisateur a déjà notés via la relation RATED.
  * WITH COLLECT(b.ISBN) AS alreadyRated :
    * Regroupe les ISBN des livres déjà évalués dans une liste alreadyRated.
    * Cette liste sera utilisée plus tard pour exclure ces livres des recommandations.
      
*  Étape 2 : Trouver les livres évalués par d'autres utilisateurs.

   MATCH (u2:user)-[r:RATED]->(b2:Book)
   WHERE NOT b2.ISBN IN alreadyRated AND u2 <> u1

  * MATCH (u2:user)-[r:RATED]->(b2:Book) :
    * Trouve tous les autres utilisateurs (u2) et les livres (b2) qu'ils ont évalués.
  * WHERE NOT b2.ISBN IN alreadyRated AND u2 <> u1 :
    * Exclut les livres déjà évalués par l'utilisateur cible (b2.ISBN NOT IN alreadyRated).
    * Ignore l'utilisateur cible lui-même (u2 <> u1).

*  Étape 3 : Préparer les évaluations des utilisateurs similaires.

   WITH u2, b2, r.rating AS rating, u1

  * WITH:
    * Passe les résultats intermédiaires à l'étape suivante :
      * u2 : Utilisateurs similaires.
      * b2 : Livres évalués par ces utilisateurs.
      * r.rating : Notes données aux livres.
        
* Étape 4 : Identifier les relations avec l'utilisateur cible.

  MATCH (u1)-[r1:RATED]->(b:Book)<-[r2:RATED]-(u2)

  * MATCH (u1)-[r1:RATED]->(b:Book)<-[r2:RATED]-(u2) :
    * Trouve les livres évalués en commun par l'utilisateur cible (u1) et les utilisateurs similaires (u2).
    * Cette étape permet de mesurer la similarité entre les utilisateurs.

* Étape 5 : Calculer les scores pour chaque livre.

  WITH b2, AVG(rating) AS AvgRating, COUNT(*) AS SimilarityScore

  * AVG(rating) :
    * Calcule la moyenne des notes données par les utilisateurs similaires pour chaque livre (b2).
    * Donne une indication de la qualité des livres selon les utilisateurs similaires.
  * COUNT(*) :
    * Compte le nombre d'utilisateurs similaires ayant évalué chaque livre.
    * Ce score (SimilarityScore) mesure la pertinence du livre en fonction de sa popularité parmi les utilisateurs similaires.

* Étape 6 : Filtrer les livres avec des meilleures notes.

  WHERE AvgRating > 7

  * WHERE AvgRating > 7:
    * Exclut les livres pour lesquels la moyenne des notes est inférieure à 7 car on considère qu'un livre est mieux noté si sa note moyenne est supérieure à 7. On rappelle que la note (rating) est comprise entre 0 et 10.

* Étape 7 : Trier et limiter les résultats.

  ORDER BY SimilarityScore DESC, AvgRating DESC
  RETURN b2.title AS RecommendedBook, b2.author AS Author, AvgRating, SimilarityScore  LIMIT 10;

  * ORDER BY SimilarityScore DESC, AvgRating DESC :
    * Trie les résultats pour privilégier les livres:
      1. Évalués par le plus grand nombre d'utilisateurs similaires (SimilarityScore élevé).
      2. Ayant une moyenne de notes élevée (AvgRating élevé).
    * LIMIT 10 :
      * Limite les résultats aux 10 livres les plus recommandés.
   